In [1]:
-- CIS 194 Homework 2

module Log where

import Control.Applicative

data MessageType = Info
                 | Warning
                 | Error Int
  deriving (Show, Eq)

type TimeStamp = Int

data LogMessage = LogMessage MessageType TimeStamp String
                | Unknown String
  deriving (Show, Eq)

data MessageTree = Leaf
                 | Node MessageTree LogMessage MessageTree
  deriving (Show, Eq)

-- | @testParse p n f@ tests the log file parser @p@ by running it
--   on the first @n@ lines of file @f@.
testParse :: (String -> [LogMessage])
          -> Int
          -> FilePath
          -> IO [LogMessage]
(testParse parse n file = take n . parse <$> readFile file

-- | @testWhatWentWrong p w f@ tests the log file parser @p@ and
--   warning message extractor @w@ by running them on the log file
--   @f@.
testWhatWentWrong :: (String -> [LogMessage]))
                  -> ([LogMessage] -> [String])
                  -> FilePath
                  -> IO [String]
testWhatWentWrong parse whatWentWrong file
  = whatWentWrong . parse <$> readFile file


Pattern-matchng is possible using both lists (inc. strings) and data types. Pattern matching is order sensitive; e.g. in a recursive function, the base case appears first. 
It's possible to nest Pattern matching. It can be aliased with the "Record Wildcards" extension + named types.


In [ ]:
parseMessage' (s:xs) = 
   case s of 
    'E' -> LogMessage (Error i) (read (head ws) :: Int) $ unwords $ tail ws
    'I' -> LogMessage Info i $ unwords ws
    'W' -> LogMessage Warning i $ unwords ws
     _  -> Unknown (s:xs)
    where 
       (w:ws) = words xs
       i = read w :: Int --indentation must be correct for multiple where vars
            
parseMessage' "E 2 562 help help" == LogMessage (Error 2) 562 "help help"
parseMessage' "I 29 la la la" == LogMessage Info 29 "la la la"

Here we use destructuring `(s:xs)` Followed by pattern matching. This is not much different from
```python
s = next(gen)
if s == 'E' ....
else: Unknown [s] + gen
```
etc. However, we get problems in python because `next` drains the generator, changing its state. Haskell's pattern matching allows us to ask questions about the data without changing state; of course, casing `s` in this case will realize the head of the sequence. 



In [2]:

g (' ':xs) = "space"
g _ = "no-space"
g "rad"
g "  riz"

get (' ':xs) n = get xs $ n + 1
get (x:xs) n   = x : get xs n
get [] n =       []

get "foo bar biz" 0


"no-space"

"space"

"foobarbiz"

In [3]:
afterNChars' :: Int -> Char -> String -> Int -> String

--Examlpe of a parameterized function. Will give the rest of the string after 
-- `stop` occurrences of character `c` in the string. The first two arguments parameterize the funciton.
afterNChars' _ _ [] _ = []
afterNChars' stop c (x:xs) n 
    | x == c    =  if n + 1 /= stop then recur xs $ n + 1 else xs
    | otherwise = recur xs n
  where recur = afterNChars' stop c

--afterNChars' 1 'X' "XfooXbarXbizXbaz" 0 
--after3Spaces = afterNChars' 3 ' ' 

msgType :: Char -> [String] -> (MessageType, Int, Int)
msgType    'E' (n:ts:xs) = (Error (read n :: Int), read ts :: Int, 3)
msgType    'W' (ts:xs)   = (Warning, read ts :: Int, 2)
msgType    'I' (ts:xs)   = (Info, read ts :: Int, 2)

parseMessage s = 
  LogMessage e ts m
  where 
    (w:ws) = words s
    (e, ts, i) = msgType (head w) ws -- tuple unpacking works like in python!
    m          = afterNChars' i ' ' s 0
    
parseMessage "E 2 562 help help" == LogMessage (Error 2) 562 "help help"
parseMessage "I 29 la la la" == LogMessage Info 29 "la la la"

-- unlike core.match, vars won't be bound (be looked up or have meaning) in pattern matches!

True

True

In [5]:
parse' = map parseMessage' . lines
insert' (Unknown _) t = t
insert' new Leaf = Node Leaf new Leaf
insert' n@(LogMessage _ nts _) (Node left c@(LogMessage _ ts _) right )
     | nts >= ts =  Node left c $ insert' n right
     | otherwise =  Node (insert' n left) c right
            
-- build $ testParse parse' 10 "error.log" 

build :: [LogMessage] -> MessageTree
build = foldl (flip insert') Leaf 

Here we introduce the `@` character, which acts like `:as` in core.matched--aliasing the entire destructured object whilst allowing pattern match/destructuring on in its innards!
We also have guards, represented by cases started with `|`.
`insert'` is a simple recursive function.

Couldn't match expected type ‘[LogMessage]’ with actual type ‘IO [LogMessage]’
In the second argument of ‘($)’, namely ‘testParse parse' 10 "error.log"’
In the expression: build $ testParse parse' 10 "error.log"


In [43]:

--testBuild build parse n file = build . take n . parse <$> readFile file

-- this is a fold
-- inOrder t = inOrder' t []  -- fip inOrder' [] 

inOrder' :: MessageTree -> [LogMessage] -> [LogMessage]
inOrder' Leaf acc = acc
inOrder' (Node Leaf e r) acc = inOrder' r (e:acc)
inOrder' (Node l e r) acc = (inOrder' l acc) ++ [e] ++ (inOrder' r acc)

-- simpler version, don't need an accumulator
inOrder Leaf = []
inOrder (Node l e r) = inOrder l ++ [e] ++ inOrder r


`inOrder'` takes a tree and returns the list representing an in-order traversal of that tree. 

In [45]:
take 3 . reverse .  inOrder . build . parse' <$> readFile "sample.log"


[LogMessage Info 11 "Initiating self-destruct sequence",LogMessage (Error 99) 10 "Flange failed!",LogMessage Info 9 "Back from lunch"]

In [40]:
-- take 10 . inOrder . build .  parse' <$> readFile "error.log"
-- build .  take 10 . parse' <$> readFile "error.log"
whatWentWrong :: [LogMessage] -> [String]

whatWentWrong = filter (/= []) . map matters . inOrder . build
 --  where matters (LogMessage  ts msg) = if ts > 50 then msg else []
      where matters (LogMessage (Error n) _ msg) = if n > 50 then msg else []
            matters _ = []

testWhatWentWrong  parse' whatWentWrong "sample.log" -- IO == ["Way too many pickles","Bad pickle-flange interaction detected","Flange failed!"]

testWhatWentWrong  parse' whatWentWrong "error.log"

-- https://www.youtube.com/watch?v=yYcFJLl6JZs&index=1&list=RDyYcFJLl6JZs
-- Mad Watch :)

["Way too many pickles","Bad pickle-flange interaction detected","Flange failed!"]

["Mustardwatch opened, please close for proper functioning!","All backup mustardwatches are busy","Depletion of mustard stores detected!","Hard drive failure: insufficient mustard","Twenty seconds remaining until out-of-mustard condition","Ten seconds remaining until out-of-mustard condition","Empty mustard reservoir! Attempting to recover...","Recovery failed! Initiating shutdown sequence"]